In [72]:
import requests
from pandas import DataFrame as df
import reverse_geocoder as rg 
import plotly.graph_objects as go

In [115]:
r = requests.get("https://coronavirus-tracker-api.herokuapp.com/v2/locations")
r = df(r.json()['locations'])

#reorganizing the fetched data

lat,lon,place= [] , [] , []
for coord in r['coordinates']:
    lat.append(coord['latitude'])
    lon.append(coord['longitude'])
    result=rg.search( (coord['latitude'],coord['longitude']) )[0]
    place.append(result['name']+" , "+result['admin1'])
    
r['latitude'],r['longitude'],r['place']= df(lat),df(lon),df(place)

confirmed,deaths,recovered = [] , [] , []
confirmed_size,deaths_size,recovered_size = [] , [] , []
for l in r['latest']:
    confirmed.append(l['confirmed'])
    deaths.append(l['deaths'])
    recovered.append(l['recovered'])
    if int(l['confirmed'])/7000 <0:
         confirmed_size.append(0)
    else:
         confirmed_size.append(int(l['confirmed'])/7000)
    
r['confirmed'],r['deaths'],r['recovered'] = df(confirmed),df(deaths),df(recovered)
r['confirmed_size'] = df(confirmed_size) 

r

,id,country,country_code,country_population,province,last_updated,coordinates,latest,latitude,longitude,place,confirmed,deaths,recovered,confirmed_size
0,0,Afghanistan,AF,29121286.0,,2020-04-16T20:25:43.530421Z,"{'latitude': '33.0', 'longitude': '65.0'}","{'confirmed': 784, 'deaths': 25, 'recovered': 0}",33.0,65.0,"Kajran , Daykundi",784,25,0,0.112000
1,1,Albania,AL,2986952.0,,2020-04-16T20:25:43.533860Z,"{'latitude': '41.1533', 'longitude': '20.1683'}","{'confirmed': 494, 'deaths': 25, 'recovered': 0}",41.1533,20.1683,"Labinot-Fushe , Elbasan",494,25,0,0.070571
2,2,Algeria,DZ,34586184.0,,2020-04-16T20:25:43.537215Z,"{'latitude': '28.0339', 'longitude': '1.6596'}","{'confirmed': 2160, 'deaths': 336, 'recovered'...",28.0339,1.6596,"I-n-Salah , Tamanghasset",2160,336,0,0.308571
3,3,Andorra,AD,84000.0,,2020-04-16T20:25:43.540571Z,"{'latitude': '42.5063', 'longitude': '1.5218'}","{'confirmed': 673, 'deaths': 33, 'recovered': 0}",42.5063,1.5218,"Andorra la Vella , Andorra la Vella",673,33,0,0.096143
4,4,Angola,AO,13068161.0,,2020-04-16T20:25:43.543945Z,"{'latitude': '-11.2027', 'longitude': '17.8739'}","{'confirmed': 19, 'deaths': 2, 'recovered': 0}",-11.2027,17.8739,"Camacupa , Bie",19,2,0,0.002714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,France,FR,64768389.0,Saint Pierre and Miquelon,2020-04-16T20:25:44.395850Z,"{'latitude': '46.8852', 'longitude': '-56.3159'}","{'confirmed': 1, 'deaths': 0, 'recovered': 0}",46.8852,-56.3159,"Saint-Pierre , Saint-Pierre",1,0,0,0.000143
260,260,South Sudan,SS,8260490.0,,2020-04-16T20:25:44.399314Z,"{'latitude': '6.877000000000001', 'longitude':...","{'confirmed': 4, 'deaths': 0, 'recovered': 0}",6.877000000000001,31.307,"Bor , Jonglei",4,0,0,0.000571
261,261,Western Sahara,EH,273008.0,,2020-04-16T20:25:44.402659Z,"{'latitude': '24.2155', 'longitude': '-12.8858'}","{'confirmed': 6, 'deaths': 0, 'recovered': 0}",24.2155,-12.8858,"Gueltat Zemmour , Laayoune-Boujdour-Sakia El H...",6,0,0,0.000857
262,262,Sao Tome and Principe,ST,197700.0,,2020-04-16T20:25:44.406008Z,"{'latitude': '0.18636', 'longitude': '6.613081'}","{'confirmed': 4, 'deaths': 0, 'recovered': 0}",0.18636,6.613081,"Sao Tome , Sao Tome Island",4,0,0,0.000571


In [116]:
#Mapping data

map_confirmed = go.Scattermapbox(
    name = "Confirmed Cases",
    lon = r["longitude"],
    lat = r['latitude'],
    text = r['place']+" , "+r['country'],
    customdata=r.loc[:, ['confirmed','deaths','recovered']],
    hovertemplate=
        "<b>%{text}</b><br><br>" +
        "Confirmed: %{customdata[0]}<br>" +
        "Deaths : %{customdata[1]}<br>"
        "<extra></extra>",
    mode = 'markers',
    showlegend=True,
    marker = go.scattermapbox.Marker(
        size = r['confirmed_size'],
        color = 'red',
        opacity = 0.5
    )
)

In [117]:
#setting up the layout

layout = go.Layout(
    mapbox_style = "carto-darkmatter",
    autosize = True,
    mapbox_layers = [
        {
            'below' : 'traces',
            'sourcetype' : 'raster',
#             'source' : ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]
        }
    ]
)

In [118]:
#Plotting the graph

data = [map_confirmed]
fig = go.Figure(data=data,layout=layout)
fig.show()